In [95]:
import numpy as np
import itertools
import random
import simpy

In [104]:
# All units are in minutes

# Simulation time in minutes
SIM_TIME = 500 * 60              

# number of docks/berth
DOCKS = 2

# number of tug/crane 
TUGS = 3

# shape parameter of Erlang
K = 3

UNLOAD_SPEED = 1
UNLOADING_TIME = 120.0

# average inter-arrival times for ships of type 1 to 3
AVG_TRAVEL_TYPE1_3 = 180.0

# average unloading time for ships of type 1
UNLOAD_TIME_TYPE_1 = UNLOADING_TIME

# average unloading time for ships of type 2
UNLOAD_TIME_TYPE_2 = UNLOADING_TIME

# average unloading time for ships of type 3
UNLOAD_TIME_TYPE_3 = UNLOADING_TIME

BERTHING_TIME = 45.0
DEBERTHING_TIME = 30.0


"""----- Ship distribution by type ----"""
# ship distribution of ships type 1 to 3 {type 1: 25%, type 2: 55%, type 3: 20%}
SHIP_DISTR = [1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,3,3,3,3]

# time takes for tug to travel each direction
TUG_TIME_ONE_WAY = 0

"""-----------LOG----------------"""
# wait crane to berthing
crane_wt1 = []
# wait for berth
berth_wt = []
# wait crane to deberthing
crane_wt2 = []

In [105]:
def ship_generator(env, tug, dock):
    #Generate new ships.
    i = 0
    while True:
        yield env.timeout(1)
        # randomly select a ship type from SHIP_DISTR
        shiptype = random.choice(SHIP_DISTR)
        env.process(ship('Ship %d' % i, env, tug, dock, shiptype))
        yield env.timeout(time_to_arrive())
        i += 1

def time_to_arrive():
    # Erlang/Gamma distribution with shape k, scale theta
    theta = AVG_TRAVEL_TYPE1_3/K
    return random.gammavariate(K,theta)
    
def time_to_berth():
    return random.expovariate(1.0/BERTHING_TIME)

def time_to_deberth():
    return random.expovariate(1.0/DEBERTHING_TIME)

def time_to_unload(shiptype):
    unloadTime = {
            1 : UNLOAD_TIME_TYPE_1,
            2 : UNLOAD_TIME_TYPE_2,
            3 : UNLOAD_TIME_TYPE_3,
                  }
    return random.expovariate(1.0/unloadTime[shiptype])

def ship(name, env, tug, dock, shiptype):
    # waiting time
    wt = 0
    
    print('%s arriving at port at %.1f' % (name, env.now))
    with tug.request() as req:
        # Request the tug
        wt = env.now
        yield req
        wt = env.now - wt

        #crane_waiting_time.
        crane_wt1.append(wt)

        print('%s waiting at port for tug at %.1f' % (name, env.now))
        # Tug transport to dock 
        yield env.process(tug_transport(name, env, tug, shiptype))
        print('%s brought to dock at %.1f' % (name, env.now))
        
        # request for dock and process ship at dock
        env.process(shipAtDock(name, env, dock, shiptype))

        
def tug_transport(name, env, tug, shiptype):
    """Arrives at the port or the dock after a 1h delay and transports the ship."""
    # print('Tug starts %s transport at %.1f' % (name, env.now))
    yield env.timeout(TUG_TIME_ONE_WAY)
    # print('Tug stops %s transport at %.1f' % (name, env.now))


def shipAtDock(name, env, dock, shiptype):
    # unload time    
    ut = 0
    with dock.request() as req:
        # Request a dock
        wt = env.now
        yield req
        wt = env.now - wt
        yield env.timeout(time_to_berth())

        #berth_waiting_time.
        berth_wt.append(wt)
        
        print('%s of shiptype %d unloading at %.1f' % (name, shiptype, env.now))
        yield env.timeout(time_to_unload(shiptype))
        print('%s unloaded at %.1f' % (name, env.now))
        
        with tug.request() as req:
            # Request the tug
            wt = env.now
            yield req
            wt = env.now - wt
            yield env.timeout(time_to_deberth())
            
            #crane_waiting_time.
            crane_wt2.append(wt)

            # Tug transport takes 1h
            print('%s waiting at dock for tug at %.1f' % (name, env.now))
            yield env.process(tug_transport(name, env, tug, shiptype))
            print('%s brought back to port at %.1f' % (name, env.now))


In [106]:
# Setup and start the simulation
print('Marine Transport Simulation')

# Create environment and start processes
env = simpy.Environment()

# ships_type_4 = simpy.Resource(env, 1)

# Resource with capacity of usage slots that can be requested by processes.
tug = simpy.Resource(env, TUGS)

# Resource with capacity of usage slots that can be requested by processes.
dock = simpy.Resource(env, DOCKS)

# Process an event yielding generator.
env.process(ship_generator(env, tug, dock))

# Execute!
env.run(until=SIM_TIME)

Marine Transport Simulation
Ship 0 arriving at port at 1.0
Ship 0 waiting at port for tug at 1.0
Ship 0 brought to dock at 1.0
Ship 0 of shiptype 2 unloading at 77.5
Ship 0 unloaded at 108.1
Ship 0 waiting at dock for tug at 123.8
Ship 0 brought back to port at 123.8
Ship 1 arriving at port at 289.0
Ship 1 waiting at port for tug at 289.0
Ship 1 brought to dock at 289.0
Ship 1 of shiptype 1 unloading at 351.4
Ship 2 arriving at port at 420.6
Ship 2 waiting at port for tug at 420.6
Ship 2 brought to dock at 420.6
Ship 2 of shiptype 3 unloading at 452.4
Ship 1 unloaded at 488.6
Ship 1 waiting at dock for tug at 534.8
Ship 1 brought back to port at 534.8
Ship 2 unloaded at 564.4
Ship 2 waiting at dock for tug at 577.2
Ship 2 brought back to port at 577.2
Ship 3 arriving at port at 731.7
Ship 3 waiting at port for tug at 731.7
Ship 3 brought to dock at 731.7
Ship 3 of shiptype 1 unloading at 776.9
Ship 3 unloaded at 824.1
Ship 3 waiting at dock for tug at 865.7
Ship 3 brought back to port 

In [107]:
print('avg. delay: ', np.mean(crane_wt1)  + np.mean(berth_wt) + np.mean(crane_wt2))
print np.mean(crane_wt1), np.mean(berth_wt), np.mean(crane_wt2)

('avg. delay: ', 20.33830061297143)
0.0 20.33830061297143 0.0
